In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from tqdm.notebook import trange
# Device configuration
device = torch.device('mps')

In [11]:
import os
os.chdir('..')

In [12]:
from torchvision.transforms import Compose, ToTensor, Normalize

_default_mnist_avalanche_transform = Compose(
    [ToTensor(), Normalize((0.1307), (0.3081))]
)
bs = 100

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=_default_mnist_avalanche_transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=_default_mnist_avalanche_transform, download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



In [13]:
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features // 2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features // 2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1)

    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))

In [14]:
from src.model.rnd.gan_generator import MNISTGanGenerator
# build network
z_dim = 100
mnist_dim = train_dataset.train_data.size(1) * train_dataset.train_data.size(2)

G = MNISTGanGenerator(input_dim=z_dim, output_dim=mnist_dim).to(device)
D = Discriminator(mnist_dim).to(device)

In [15]:
D

Discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)

In [16]:
# loss
criterion = nn.BCELoss()

# optimizer
lr = 0.0002
G_optimizer = optim.Adam(G.parameters(), lr=lr)
D_optimizer = optim.Adam(D.parameters(), lr=lr)

In [17]:
def D_train(x):
    #=======================Train the discriminator=======================#
    D.zero_grad()

    # train discriminator on real
    x_real, y_real = x.view(-1, mnist_dim), torch.ones(bs, 1)
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

    D_output = D(x_real)
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    # train discriminator on facke
    z = Variable(torch.randn(bs, z_dim).to(device))
    x_fake, y_fake = G(z), Variable(torch.zeros(bs, 1).to(device))

    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()

    return D_loss.data.item()

In [18]:
def G_train(x):
    #=======================Train the generator=======================#
    G.zero_grad()

    z = Variable(torch.randn(bs, z_dim).to(device))
    y = Variable(torch.ones(bs, 1).to(device))

    G_output = G(z)
    D_output = D(G_output)
    G_loss = criterion(D_output, y)

    # gradient backprop & optimize ONLY G's parameters
    G_loss.backward()
    G_optimizer.step()

    return G_loss.data.item()

In [19]:
n_epoch = 100
for epoch in trange(1, n_epoch + 1):
    D_losses, G_losses = [], []
    for batch_idx, (x, _) in enumerate(train_loader):
        D_losses.append(D_train(x))
        G_losses.append(G_train(x))

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
        (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))

  0%|          | 0/100 [00:00<?, ?it/s]

[1/100]: loss_d: 1.378, loss_g: 0.680
[2/100]: loss_d: 1.326, loss_g: 0.669
[3/100]: loss_d: 1.232, loss_g: 0.623
[4/100]: loss_d: 1.114, loss_g: 0.566
[5/100]: loss_d: 0.983, loss_g: 0.503
[6/100]: loss_d: 0.850, loss_g: 0.441
[7/100]: loss_d: 0.724, loss_g: 0.383
[8/100]: loss_d: 0.609, loss_g: 0.331
[9/100]: loss_d: 0.509, loss_g: 0.287
[10/100]: loss_d: 0.423, loss_g: 0.249
[11/100]: loss_d: 0.352, loss_g: 0.218
[12/100]: loss_d: 0.293, loss_g: 0.191
[13/100]: loss_d: 0.246, loss_g: 0.169
[14/100]: loss_d: 0.208, loss_g: 0.149
[15/100]: loss_d: 0.177, loss_g: 0.133
[16/100]: loss_d: 0.152, loss_g: 0.118
[17/100]: loss_d: 0.131, loss_g: 0.106
[18/100]: loss_d: 0.114, loss_g: 0.095
[19/100]: loss_d: 0.099, loss_g: 0.085
[20/100]: loss_d: 0.087, loss_g: 0.077
[21/100]: loss_d: 0.077, loss_g: 0.069
[22/100]: loss_d: 0.068, loss_g: 0.062
[23/100]: loss_d: 0.061, loss_g: 0.056
[24/100]: loss_d: 0.054, loss_g: 0.051
[25/100]: loss_d: 0.049, loss_g: 0.046
[26/100]: loss_d: 0.044, loss_g: 0

In [21]:
with torch.no_grad():
    test_z = Variable(torch.randn(bs, z_dim).to(device))
    generated = G(test_z)

    save_image(generated.view(generated.size(0), 1, 28, 28), 'sample_' + '.png')